In [1]:
import carla
import math
import random
import time
import numpy as np


In [2]:
import cv2

In [3]:
client = carla.Client('localhost', 2000)
world = client.get_world()

In [4]:
client.load_world('Town01')

In [5]:
bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

In [6]:
spawn_points

In [7]:
map = world.get_map()


In [8]:
map

In [5]:
#creating a vechile
vehicle_bp = bp_lib.find('vehicle.lincoln.mkz_2020')
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

In [6]:
#3shan nela2y el car badl mndwr bel mouse wel wasd: (location above(z=2.5) and behind(x=-4) the car)
spectator = world.get_spectator()
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)), vehicle.get_transform().rotation)
spectator.set_transform(transform)

In [8]:
#n3ml 30 vehicles tanyen:
for i in range(80):
    vehicle_bp = random.choice(bp_lib.filter('vehicle')) #randomly select vehicles from blueprint library
    npc = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

In [9]:
#set all vehicles in motion using carla traffic manager:
for v in world.get_actors().filter('*vehicle*'):
    v.set_autopilot(True)

In [10]:
def process_img(image):
    i = np.array(image.raw_data)  # convert to an array
    i2 = i.reshape((IM_HEIGHT, IM_WIDTH, 4))  # was flattened, so we're going to shape it.
    i3 = i2[:, :, :3]  # remove the alpha (basically, remove the 4th index  of every pixel. Converting RGBA to RGB)
    cv2.imshow("", i2)  # show it.
    cv2.waitKey(1)
    return i3/255.0  # normalize

In [11]:
IM_WIDTH = 640
IM_HEIGHT = 480

In [13]:

cam_bp = bp_lib.find("sensor.camera.rgb")
# change the dimensions of the image
cam_bp.set_attribute('image_size_x', f'{IM_WIDTH}')
cam_bp.set_attribute('image_size_y', f'{IM_HEIGHT}')
cam_bp.set_attribute('fov', '110')

# Adjust sensor relative to vehicle
spawn_point = carla.Transform(carla.Location(z=2))

# spawn the sensor and attach to vehicle.
sensor = world.spawn_actor(cam_bp, spawn_point, attach_to=vehicle)

sensor.listen(lambda data: process_img(data))


In [12]:
#record data with rgb camera
camera_bp = bp_lib.find('sensor.camera.rgb')
camera_init_trans = carla.Transform(carla.Location(z=2)) #mkan el camera(fo2 el vehicle)
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to = vehicle) #spawn el camera attached to awl vehicle ana 3mltha

In [13]:
#lambda->start recording
camera.listen(lambda image: image.save_to_disk('out/%06d.png' % image.frame))

In [14]:
camera.stop()